In [5]:
from shapely.geometry import Polygon, MultiPolygon, asShape
from shapely.ops import unary_union, cascaded_union
from geojson import Feature
import json
import pandas as pd
import os

In [3]:
data_folder = 'data'

In [72]:
j = json.load(open(os.path.join(data_folder, "chicago-community-areas.json")))

In [73]:
df = pd.read_excel(os.path.join(data_folder, 'Chicago-Communities.xlsx'))
df.Community = df.Community.apply(lambda x: str.upper(x).strip())
df.head()

,Community,District
0,ROGERS PARK,Far North Side
1,WEST RIDGE,Far North Side
2,UPTOWN,Far North Side
3,LINCOLN SQUARE,Far North Side
4,EDISON PARK,Far North Side


In [74]:
districts_areas = df.groupby('District')['Community'].apply(list)

In [77]:
# get indices of areas for each district
indices=[]
for k, da in enumerate(districts_areas):
    indices.append([])
    indices[k] = [idx for idx, i in enumerate(j['features']) if \
        i['properties']['community'] in da]

In [79]:
# create polygons for areas
all_polygons = []
for row in indices:
    all_polygons.append([asShape(j['features'][i]['geometry']) for i in row])

In [81]:
# Add district name as information
properties=[]
for k,row in enumerate(indices):
    prop = j['features'][row[0]]['properties']
    prop['district'] = districts_areas.index[k]
    properties.append(prop)

In [82]:
# join polygons
joined = []
for polygons in all_polygons:
    joined.append(unary_union(polygons))

In [83]:
# delete the merged counties
d = j
for i in reversed(sorted(list((x for y in indices for x in y)))):
      del d['features'][i]

In [84]:
# add the new polygon to the features
for k,join in enumerate(joined):
    feature = Feature(geometry=join, properties=properties[k])
    d['features'].append(feature)

In [85]:
# save the geojson
with open(os.path.join(data_folder,'chicago-district-boundaries.json'), 'w') as out:
  json.dump(d, out)